# Modelo final ejecutable – Vehicle Energy Dataset

Este notebook carga el **modelo final de combustión** entrenado (XGBoost) 
y genera predicciones de consumo de combustible en L/100km para un archivo 
CSV de entrada con el mismo formato de features que se usó durante el 
desarrollo del modelo (`X_metrics_processed.csv`).

**Uso:**

1. Ajustar la ruta del archivo de entrada (`INPUT_CSV_PATH`) en la última celda.
2. Ejecutar todas las celdas.
3. El notebook generará un archivo CSV de salida con la columna  `Y_consumption_combustion_L_per_100km_pred`.

In [1]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd


In [2]:
# Directorios base (ajustar si tu estructura es distinta)
PROJECT_DIR = Path(".").resolve()
DATA_DIR = PROJECT_DIR / "Data"
MODELS_DIR = PROJECT_DIR / "models" / "saved_models" / "final"

print("PROJECT_DIR:", PROJECT_DIR)
print("MODELS_DIR:", MODELS_DIR)

# ----------------------------
# Cargar artefactos finales
# ----------------------------

# Estos nombres vienen de 5final_model_train_test.ipynb
model_path = MODELS_DIR / "xgboost_combustion.pkl"
scaler_path = MODELS_DIR / "scaler_combustion.pkl"
feat_path = MODELS_DIR / "feature_names.pkl"

if not model_path.exists():
    raise FileNotFoundError(f"No se encontró el modelo de combustión: {model_path}")

if not scaler_path.exists():
    raise FileNotFoundError(f"No se encontró el scaler de combustión: {scaler_path}")

if not feat_path.exists():
    raise FileNotFoundError(f"No se encontró feature_names.pkl: {feat_path}")

model_combustion = joblib.load(model_path)
scaler_combustion = joblib.load(scaler_path)
feature_names = joblib.load(feat_path)  # lista de columnas con nombres "sanitizados"

print("\nArtefactos cargados correctamente:")
print(f"- Modelo: {model_path.name}")
print(f"- Scaler: {scaler_path.name}")
print(f"- n_features: {len(feature_names)}")


def build_feature_matrix(df: pd.DataFrame, feature_names):
    """
    Alinea el DataFrame de entrada con las columnas usadas para entrenar el modelo.

    - Toma solo las columnas presentes en feature_names.
    - Si falta alguna columna necesaria, levanta un error claro.
    - Convierte a numérico y hace imputación simple de NaN con la media.
    """
    df_cols = set(df.columns)
    feat_set = set(feature_names)

    missing = sorted(feat_set - df_cols)
    if missing:
        raise ValueError(
            "El archivo de entrada NO tiene todas las columnas necesarias.\n"
            "Faltan las siguientes columnas:\n"
            f"  {missing}\n\n"
            "Revisá que el CSV de entrada tenga el mismo formato que "
            "el usado en el entrenamiento (X_metrics_processed.csv)."
        )

    # Nos quedamos exactamente con las columnas en el orden esperado
    X = df.loc[:, feature_names].copy()

    # Nos aseguramos de que todo sea numérico
    X = X.apply(pd.to_numeric, errors="coerce")

    # Imputación simple: NaN -> media de la columna
    X = X.fillna(X.mean())

    return X


def run_inference_combustion(input_csv: Path, output_csv: Path | None = None):
    """
    Genera predicciones de consumo de combustión (L/100km) para un CSV de entrada.

    - input_csv: ruta al archivo de entrada (mismo formato de features que X_metrics_processed).
    - output_csv: ruta de salida. Si es None, se crea un archivo con sufijo '_predictions'.
    """
    input_csv = Path(input_csv)

    if not input_csv.exists():
        raise FileNotFoundError(f"No se encontró el archivo de entrada: {input_csv}")

    print(f"\nLeyendo archivo de entrada: {input_csv}")
    df = pd.read_csv(input_csv)
    print(f"Filas: {len(df)}, Columnas: {len(df.columns)}")

    print("\nConstruyendo matriz de features...")
    X = build_feature_matrix(df, feature_names)

    print("Aplicando scaler...")
    X_scaled = scaler_combustion.transform(X)

    print("Generando predicciones de consumo (L/100km)...")
    y_pred = model_combustion.predict(X_scaled)
    y_pred = np.asarray(y_pred).reshape(-1)

    pred_col = "Y_consumption_combustion_L_per_100km_pred"
    df[pred_col] = y_pred

    if output_csv is None:
        output_csv = input_csv.with_name(input_csv.stem + "_predictions.csv")
    else:
        output_csv = Path(output_csv)

    output_csv.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_csv, index=False)

    print(f"\nArchivo de salida guardado en: {output_csv}")
    print("Primeras filas con predicciones:")
    display(df[[pred_col]].head())

    return df


PROJECT_DIR: C:\Users\joaco\OneDrive\Documents\facultad\ML\VED-TPF-ML-Fadel-DiCola
MODELS_DIR: C:\Users\joaco\OneDrive\Documents\facultad\ML\VED-TPF-ML-Fadel-DiCola\models\saved_models\final

Artefactos cargados correctamente:
- Modelo: xgboost_combustion.pkl
- Scaler: scaler_combustion.pkl
- n_features: 58
